# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
# Load the csv exported in Part I to a DataFrame and read it
file = 'city_data.csv'
city_data_df = pd.read_csv(file)
city_data_df.head()

Unnamed: 0          City      Lat       Lng  Max Temp  Humidity  \
0           0         ancud -41.8697  -73.8203     42.01       100   
1           1  port lincoln  -7.0349   29.7638     67.35        76   
2           2      luderitz -26.6481   15.1594     68.52        41   
3           3        kupang -10.1667  123.5833     87.80        58   
4           4        albany  42.6001  -73.9662     51.01        75   

   Cloudiness  Wind Speed Country        Date  
0          20        3.44      CL  1620702783  
1          50        6.40      CD  1620702783  
2           0        4.21     NaN  1620702784  
3          20       18.41      ID  1620702784  
4          87        5.84      US  1620702772

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# Configure gmaps. Access maps with unique API key
gmaps.configure(api_key=g_key)

In [13]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"].astype(float)
max_humidity = humidity.max()

In [14]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
# Narrow down the cities to fit weather conditions.
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
narrow_down_df = city_data_df.loc[(city_data_df["Max Temp"] > 70) & (city_data_df["Max Temp"] < 80) & (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] == 0), :]

# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
narrow_down_df = narrow_down_df.dropna(how='any')
narrow_down_df.reset_index(inplace=True)
del narrow_down_df['index']
narrow_down_df.head()

Unnamed: 0       City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0          12       dali  25.7000  100.1833     71.62        41           0   
1         210    olimpia -20.7372  -48.9147     73.40        49           0   
2         315       jalu  29.0331   21.5482     72.61        33           0   
3         452  ramhormoz  31.2800   49.6035     77.00        41           0   
4         464  abu dhabi  24.6000   72.7000     75.49        48           0   

   Wind Speed Country        Date  
0        6.53      CN  1620702786  
1        2.30      BR  1620702857  
2        8.48      LY  1620702893  
3        9.22      IR  1620702944  
4        7.72      IN  1620702947

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
# Store into variable named hotel_df.
hotel_df = pd.DataFrame(narrow_down_df, columns=["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"])

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df.head()

City Country      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0       dali      CN  25.7000  100.1833     71.62        41           0   
1    olimpia      BR -20.7372  -48.9147     73.40        49           0   
2       jalu      LY  29.0331   21.5482     72.61        33           0   
3  ramhormoz      IR  31.2800   49.6035     77.00        41           0   
4  abu dhabi      IN  24.6000   72.7000     75.49        48           0   

   Wind Speed Hotel Name  
0        6.53             
1        2.30             
2        8.48             
3        9.22             
4        7.72

In [29]:
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.
# find the closest hotel of each type to coordinates

params = {
    "radius": 5000,
    "keyword": "hotel",
    "types": "hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    try:
    # make request and print url
        requested = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
        converted = requested.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    
        hotel_df.loc[index, "Hotel Name"] = converted["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.


In [30]:
# Drop rows with missing data
hotel_df.dropna(how='any')
# Save Data to csv
hotel_df.to_csv("Hotels_Output.csv")

# Visualize to confirm hotel data appears
hotel_df.head(10)

City Country      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0       dali      CN  25.7000  100.1833     71.62        41           0   
1    olimpia      BR -20.7372  -48.9147     73.40        49           0   
2       jalu      LY  29.0331   21.5482     72.61        33           0   
3  ramhormoz      IR  31.2800   49.6035     77.00        41           0   
4  abu dhabi      IN  24.6000   72.7000     75.49        48           0   
5     gurlan      UZ  41.8447   60.3919     71.60        30           0   
6      farah      AF  32.5000   62.5000     70.21        41           0   

   Wind Speed                    Hotel Name  
0        6.53                 Sunyata Hotel  
1        2.30  Wyndham Olímpia Royal Hotels  
2        8.48                    Jalu Hotel  
3        9.22        مجتمع رفاهی شهید رجایی  
4        7.72                 Hotel Hillock  
5        6.91                     ÒJIQ OVUL  
6        6.64

In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
temp = hotel_df["Max Temp"].astype(float)
# Display figure
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=temp, dissipating=False, point_radius=2)

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))